## Notebook to run interaction *cis*-eQTL interaction analysis using [tensorQTL](https://github.com/broadinstitute/tensorqtl)

[Taylor-Weiner, Aguet, et al., Genome Biol. 20:228, 2019.](https://genomebiology.biomedcentral.com/articles/10.1186/s13059-019-1836-7)

In [ ]:
!date

#### import libraries and set notebook variables

In [ ]:
import pandas as pd
import torch
import dask.dataframe as dd
import tensorqtl.tensorqtl as tensorqtl
from tensorqtl.tensorqtl import genotypeio, cis, trans
print('PyTorch {}'.format(torch.__version__))
print('Pandas {}'.format(pd.__version__))

import os

import warnings
warnings.filterwarnings('ignore')

In [ ]:
# parameters
cohort = 'foundin'
version = 'amppdv1'
day = 'da65'

In [ ]:
# naming
cohort_version = f'{cohort}.{version}'
cohort_build = f'{cohort}.{day}'

# directories
wrk_dir = f'/home/jupyter/{cohort}/eqtl'
geno_dir = f'{wrk_dir}/genotypes'
expr_dir = f'{wrk_dir}/expression'
info_dir = f'{wrk_dir}/sample_info'
tensorqtl_dir = f'{wrk_dir}/tensorqtl'
results_dir = f'{wrk_dir}/results'

# input files
expr_bed_file = f'{expr_dir}/{cohort_build}.norm.adj.bed.gz'
bfile_prefix_path = f'{geno_dir}/{cohort_version}.bfile'
assay_covs_files = f'{info_dir}/foundin_rnab_sample_info.csv'

# output files
used_samples_list_file = f'{info_dir}/{cohort_build}.eqtl.samples'

# constant values
alpha_value = 0.05
min_nominal_alpha = 1e-05

# cell_types_list = ['DAn', 'DopaminergicNeurons', 'EarlyneuronProgenitor',
#                    'ImmatureDopaminergicNeurons', 'ProliferatingFloorPlateProgenitors',
#                    'LateneuronProgenitor', 'Ependymal-likeCells',
#                    'Neuroepithelial-likeCells']
cell_types_list = ['DAn']
repeated_samples_to_exclude = [f'RNAB_PPMI3966B1_2813_{day}_v1', f'RNAB_PPMI3966B2_2813_{day}_v2',
                               f'RNAB_PPMI3966B5_2813_{day}_v1', f'RNAB_PPMI3966B2_2813_{day}_v1',
                               f'RNAB_PPMI3966B1_2813_{day}_v2', f'RNAB_PPMI3966B1_2813_{day}_v3',
                               f'RNAB_PPMI3966B1_2813_{day}_v4', f'RNAB_PPMI3966B1_2813_{day}_v5',
                               f'RNAB_PPMI3966B1_2813_{day}_v6', f'RNAB_PPMI3966B1_2813_{day}_v7',
                               f'RNAB_PPMI3966B1_2813_{day}_v8', f'RNAB_PPMI3966B1_2813_{day}_v9']

In [ ]:
os.makedirs(tensorqtl_dir, exist_ok=True)
os.makedirs(results_dir, exist_ok=True)

#### load phenotypes and covariates (if needed)

In [ ]:
%%time

phenotype_df, phenotype_pos_df = tensorqtl.read_phenotype_bed(expr_bed_file)
covs_df = pd.read_csv(assay_covs_files, index_col=0)
print(f'phenotype_df {phenotype_df.shape}')
print(f'phenotype_pos_df {phenotype_pos_df.shape}')
print(f'covariates_df {covs_df.shape}')
# display(phenotype_df.head())
# display(phenotype_pos_df.head())
# display(covs_df.head())

In [ ]:
phenotype_df.columns[phenotype_df.columns.str.startswith('PPMI3966')]

#### load plink bfiles

In [ ]:
%%time

# pr = genotypeio.PlinkReader(bfile_prefix_path, select_samples=phenotype_df.columns)
pr = genotypeio.PlinkReader(bfile_prefix_path)
genotype_df = pr.load_genotypes()
variant_df = pr.bim.set_index('snp')[['chrom', 'pos']]

In [ ]:
print(genotype_df.shape)
# display(genotype_df.head())
print(variant_df.shape)
# display(variant_df.head())

In [ ]:
# tensorQTL says wants plink bfiles, but wants bim chrs to include 'chr'
variant_df['chrom'] = 'chr' + variant_df['chrom']
print(variant_df.shape)
# display(variant_df.head())

#### make sure the pheno and genos have same samples

In [ ]:
assay_intersect_samples = set(genotype_df.columns) & set(phenotype_df.columns) 
print(f'intersect {len(assay_intersect_samples)}')
extra_geno_samples = set(genotype_df.columns) - set(phenotype_df.columns)
print(f'number of genotypes samples not in expression {len(extra_geno_samples)}')
extra_expr_samples = set(phenotype_df.columns) - set(genotype_df.columns)
print(f'number of expression samples not in genotypes {len(extra_geno_samples)}')

# save the used sample list
pd.DataFrame(data=assay_intersect_samples).to_csv(used_samples_list_file, 
                                                  index=False, header=False)

#### drop the non-matched samples

In [ ]:
genotype_df.drop(columns=extra_geno_samples, inplace=True)
phenotype_df.drop(columns=extra_expr_samples, inplace=True)

print(genotype_df.shape)
# display(genotype_df.head())
print(phenotype_df.shape)
# display(phenotype_df.head())

#### need to make sure phenos and genos have matched chromosomes; ie just autosomes

In [ ]:
# need to ditch any non-autosomal genes
assay_intersect_chroms = set(phenotype_pos_df['chr']) & set(variant_df['chrom']) 
print(f'intersect {len(assay_intersect_chroms)}')
extra_geno_chroms = set(variant_df['chrom']) - set(phenotype_pos_df['chr'])
print(f'number of genotypes chroms not in expression {len(extra_geno_chroms)}')
print(extra_geno_chroms)
extra_expr_chroms = set(phenotype_pos_df['chr']) - set(variant_df['chrom'])
print(f'number of expression chroms not in genotypes {len(extra_expr_chroms)}')
print(extra_expr_chroms)

In [ ]:
if len(extra_geno_chroms) > 0:
    variant_df = variant_df.loc[~variant_df['chrom'].isin(extra_geno_chroms)]
    # this will remove variants so need to remove them from genos df as well
    genotype_df = genotype_df.loc[genotype_df.index.isin(variant_df.index)]
if len(extra_expr_chroms) > 0:
    phenotype_pos_df = phenotype_pos_df.loc[~phenotype_pos_df['chr'].isin(extra_expr_chroms)]
    # this will remove genes so need to remove them from phenos df as well
    phenotype_df = phenotype_df.loc[phenotype_df.index.isin(phenotype_pos_df.index)]

print(genotype_df.shape)
# display(genotype_df.head())
print(variant_df.shape)
# display(variant_df.head())
print(phenotype_df.shape)
# display(phenotype_df.head())
print(phenotype_pos_df.shape)
# display(phenotype_pos_df.head())

#### make sure covariates match geno and pheno samples

In [ ]:
# subest covs to just this 'day'; ie all differention days covs in file
# also since only interested in cell fractions as interaction terms, subset now
covs_df = covs_df.loc[(covs_df['day'] == day) & (covs_df['sampleid'].isin(phenotype_df.columns))]
covs_df = covs_df.loc[~covs_df.index.isin(repeated_samples_to_exclude)]
print(covs_df.shape)

cov_intersect_samples = set(phenotype_df.columns) & set(covs_df['sampleid']) 
print(f'intersect {len(cov_intersect_samples)}')
extra_expr_samples = set(phenotype_df.columns) - set(covs_df['sampleid'])
print(f'number of expression samples not in covariates {len(extra_expr_samples)}')
extra_cov_samples = set(covs_df['sampleid']) - set(phenotype_df.columns)
print(f'number of covariate samples not in genotypes {len(extra_cov_samples)}')

#### run the cell-type fraction interactions; ie the cs-eQTL or ieQTL

In [ ]:
%%time
# subset just the cell fraction covariates
keep_cols = cell_types_list + ['sampleid']
cf_covs_df = covs_df[keep_cols]
cf_covs_df.set_index('sampleid', inplace=True)
print(cf_covs_df.shape)
# display(cf_covs_df.head())

for cell_type in cell_types_list:
    cf_covs_df[cell_type].fillna(cf_covs_df[cell_type].mean(), inplace=True)

In [ ]:
%%time
for cell_type in cell_types_list:
    print(cell_type)
    cis.map_nominal(genotype_df, variant_df, phenotype_df, phenotype_pos_df, 
                    covariates_df=pd.DataFrame(index=phenotype_df.columns), 
                    interaction_s=cf_covs_df[cell_type],
                    prefix=f'{cohort_build}.{cell_type}', output_dir=tensorqtl_dir,
                    run_eigenmt=True, write_top=True, write_stats=True)

# covariates_df=pd.DataFrame(index=phenotype_df.columns),

#### load cell-type interaction top results and save significant results

In [ ]:
%%time
for cell_type in cell_types_list:
    print(cell_type)
    cell_top_file = f'{tensorqtl_dir}/{cohort_build}.{cell_type}.cis_qtl_top_assoc.txt.gz'
    cell_top_results_file = f'{results_dir}/{cohort_build}.{cell_type}.cis_qtl_top_assoc.txt.gz'
    top_cieqtl = pd.read_csv(cell_top_file, sep='\t')
    print(top_cieqtl.shape)
    top_sig_cieqtl = top_cieqtl.loc[top_cieqtl['pval_adj_bh'] <= alpha_value]
    print(top_sig_cieqtl.shape)
    top_sig_cieqtl.to_csv(cell_top_results_file)

#### load all the interaction results

In [ ]:
this_cell_type = 'DopaminergicNeurons'
import dask.dataframe as dd

cell_files = f'{tensorqtl_dir}/{cohort_build}.{this_cell_type}.cis_qtl_pairs.chr*.parquet'
cieqtl_df = dd.read_parquet(cell_files)

In [ ]:
print(cieqtl_df.shape)
display(cieqtl_df.head())